In [27]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix         
# Custom config and class labels
from models.config import (
    MODEL_PATHS,
    IMG_SIZE,
    CLASS_NAMES,
    FEATURE_STORE_PATH
)
from keras.layers import TFSMLayer


In [28]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize if used during training
    return img_array

def predict_and_log(img_path):
    # Use absolute path for loading the model
    model_path = os.path.abspath('weights/Meso4_DF.keras')

    # Load the model using the absolute path
    model = tf.keras.models.load_model(model_path)

    # Preprocess the image
    img_array = preprocess_image(img_path)

    # Make predictions using the loaded model
    prediction = model.predict(img_array)

    # Ensure the predicted label and confidence are correctly determined
    predicted_label = np.argmax(prediction)
    predicted_class = CLASS_NAMES[predicted_label]
    confidence = prediction[predicted_label]

    print(f"[INFO] Predicted Class: {predicted_class} with Confidence: {confidence:.2f}")

    # Append to FEATURE_STORE_PATH
    record = {
        "timestamp": datetime.now().isoformat(),
        "image_path": img_path,
        "predicted_class": predicted_class,
        "confidence": confidence
    }

    df = pd.DataFrame([record])
    df.to_csv(FEATURE_STORE_PATH, mode='a', header=not os.path.exists(FEATURE_STORE_PATH), index=False)
    print(f"[INFO] Prediction saved to: {FEATURE_STORE_PATH}")

    # Visualization
    plt.bar(CLASS_NAMES, prediction, color=['blue', 'orange'])
    plt.title('Prediction Confidence')
    plt.xlabel('Class')
    plt.ylabel('Confidence')
    plt.show()


In [ ]:
def evaluate_model(test_data_dir):
    # Load the model using the absolute path
    model_path = os.path.abspath('Dataset/model_result/mesonet_model.keras')
    model = tf.keras.models.load_model(model_path)

    # Load test images and labels
    test_images = []
    test_labels = []

    for class_name in CLASS_NAMES:
        class_dir = os.path.join(test_data_dir, class_name)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            img_array = preprocess_image(img_path)
            test_images.append(img_array)
            test_labels.append(CLASS_NAMES.index)
            